In [3]:
!pip install pycaret

     |████████████████████████████████| 264 kB 4.0 MB/s 
     |████████████████████████████████| 167 kB 52.4 MB/s 
     |████████████████████████████████| 14.6 MB 113 kB/s 
     |████████████████████████████████| 1.3 MB 50.1 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 6.8 MB 56.7 MB/s 
     |████████████████████████████████| 111 kB 60.9 MB/s 
     |████████████████████████████████| 80 kB 9.1 MB/s 
     |████████████████████████████████| 2.0 MB 39.4 MB/s 
     |████████████████████████████████| 1.7 MB 43.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 248 kB 63.3 MB/s 
     |████████████████████████████████| 271 kB 47.8 MB/s 
     |████████████████████████████████| 3.1 MB 50.2 MB/s 
     |████████████████████████████████| 675 kB 36.7 MB/s 
     |██████████████████

In [4]:
!pip install catboost

     |████████████████████████████████| 67.4 MB 30 kB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
from pycaret.regression import *
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler

import matplotlib.pyplot as plt
import matplotlib

import platform
if platform.system() == 'Windows': 
    matplotlib.rc('font', family='Malgun Gothic') 
elif platform.system() == 'Darwin': # Mac 
    matplotlib.rc('font', family='AppleGothic') 
else:
    matplotlib.rc('font', family='NanumGothic')
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.style.use('ggplot')

In [32]:
train_df = pd.read_csv('/content/drive/MyDrive/bigcon/final_final_train_0913.csv',index_col=0)

In [33]:
train_df

,base_date,emd_nm,em_g,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop
0,2018-01-01,전체,85755000,679922,535888.570169,3.786578e+09,20891720.0,1.602637e+09,108983.0,1057.0,40848.0,32512,7.210523e+06,8.301891e+05,54466.952499,2215.223209
1,2018-01-02,전체,90217650,679922,534509.814556,3.406059e+09,15597155.0,1.286966e+09,107164.0,839.0,38738.0,34664,6.761153e+06,1.368438e+06,53973.690842,2221.350506
2,2018-01-03,전체,80277600,679922,532472.061474,3.499232e+09,13453350.0,1.292679e+09,109765.0,728.0,38284.0,31025,6.770127e+06,1.369782e+06,53705.165769,2172.085577
3,2018-01-04,전체,80138700,679922,514678.201983,3.329829e+09,14592029.0,1.215080e+09,107110.0,740.0,36878.0,31540,6.993326e+06,1.351504e+06,53151.118467,2111.044522
4,2018-01-05,전체,75798600,679922,527383.565802,3.733755e+09,21105231.0,1.290936e+09,112064.0,1009.0,37686.0,29709,6.825653e+06,1.317354e+06,53360.869746,2055.834760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50881,2021-06-30,표선면,1377450,12681,10137.380983,9.063287e+07,1855783.0,3.831039e+07,1336.0,75.0,423.0,370,1.614560e+05,2.405863e+04,914.664667,12.910869
50882,2021-06-30,한경면,973850,9495,7257.895822,4.526361e+07,697661.0,1.767975e+07,1286.0,40.0,423.0,269,1.041218e+05,1.633182e+04,608.427552,22.165953
50883,2021-06-30,한림읍,3298100,24462,20632.727912,1.301568e+08,5508665.0,4.207137e+07,3044.0,257.0,669.0,870,3.018026e+05,5.060204e+04,2064.041876,48.329788
50884,2021-06-30,화북동,3749600,24630,24117.268821,1.918163e+08,13370520.0,4.846083e+07,5644.0,644.0,1645.0,2255,2.433203e+05,5.097469e+04,1810.431884,18.483512


# Summary
- Auto ML을 다룬 데이콘 게시글에서는 대부분 변수를 추가로 만들지 않았기 때문에 feature1에서 feature selection로만 돌려본 다음 파생변수를 추가하여 튜닝한 결과와 비교해 볼 것.
- 데이콘에서 코드에 비해 Boosting, Stacking, Bagging 등의 기법을 추가

# 모델 생성 후 성능 비교

In [34]:
df = train_df[train_df['emd_nm'] == '건입동'].set_index('base_date')
df.drop(columns=['emd_nm'],inplace=True)
df = df.reset_index(drop=True).reset_index()

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.loc[:, [col for col in df.columns if col != 'em_g']], df['em_g'], test_size=0.2, random_state=0)
train = pd.concat([X_train, y_train], axis = 1)

In [36]:
clf = setup(data=df,
            target='em_g',
            silent=True,
            fold = 5)

,Description,Value
0,session_id,8358
1,Target,em_g
2,Original Data,"(1276, 15)"
3,Missing Values,False
4,Numeric Features,14
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(893, 13)"


In [37]:
best_3 = compare_models(sort = 'RMSE', n_select = 3, fold=5)
print(best_3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,75657.7990,9.730039e+09,98178.7160,0.8548,0.0800,0.0572,4.424
et,Extra Trees Regressor,79128.6623,1.037269e+10,101542.7947,0.8442,0.0781,0.0582,0.608
gbr,Gradient Boosting Regressor,77903.7195,1.034820e+10,101547.4070,0.8437,0.0763,0.0569,0.242
rf,Random Forest Regressor,82017.5816,1.123807e+10,105689.6288,0.8318,0.0797,0.0602,0.818
lightgbm,Light Gradient Boosting Machine,81795.9268,1.181387e+10,108030.6924,0.8256,0.0904,0.0625,0.208
llar,Lasso Least Angle Regression,93113.3953,1.407088e+10,118349.1866,0.7879,0.0887,0.0683,0.020
br,Bayesian Ridge,93015.9077,1.407447e+10,118353.1244,0.7880,0.0888,0.0683,0.018
en,Elastic Net,93122.8625,1.407273e+10,118358.4219,0.7878,0.0887,0.0683,0.024
lasso,Lasso Regression,93124.3938,1.407289e+10,118359.1922,0.7878,0.0887,0.0683,0.024
ridge,Ridge Regression,93124.3766,1.407289e+10,118359.1922,0.7878,0.0887,0.0683,0.018


[<catboost.core.CatBoostRegressor object at 0x7ff7ad28d510>, ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=-1, oob_score=False,
                    random_state=8358, verbose=0, warm_start=False), GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
             

# 모델 튜닝 후 성능 비교

In [38]:
tuned_models = []

## 1. 단일 모델
catboost / Gradient Boosting / LGBM

In [39]:
# extra tree
model_01 = create_model('et')
model_01 = tune_model(model_01, fold= 5, n_iter = 100, optimize = 'RMSE', verbose = True, choose_better = True)
tuned_models.append(model_01)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,82828.7111,1.243064e+10,111492.7694,0.8442,0.0716,0.0557
1,89515.6810,1.222044e+10,110546.0972,0.8156,0.0874,0.0664
2,74007.8043,8.557622e+09,92507.4146,0.8217,0.0664,0.0536
3,84186.4638,1.142470e+10,106886.3704,0.8081,0.0841,0.0630
4,82767.1891,1.298527e+10,113952.9353,0.8452,0.1213,0.0722
Mean,82661.1698,1.152373e+10,107077.1174,0.8270,0.0862,0.0622
SD,4984.5941,1.565349e+09,7630.4823,0.0151,0.0192,0.0068


In [40]:
# Gradient Boosting Classification
model_02 = create_model('rf')
model_02 = tune_model(model_02, fold= 5, n_iter = 100, optimize = 'RMSE', verbose = True, choose_better = True)
tuned_models.append(model_02)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,83104.5112,1.274732e+10,112904.0228,0.8402,0.0723,0.0559
1,86283.9903,1.144657e+10,106988.6382,0.8273,0.0840,0.0637
2,73897.9425,8.419350e+09,91757.0137,0.8246,0.0658,0.0534
3,84056.6800,1.136555e+10,106609.3251,0.8091,0.0765,0.0609
4,86665.0703,1.368874e+10,116998.8875,0.8368,0.1216,0.0744
Mean,82801.6389,1.153350e+10,107051.5775,0.8276,0.0840,0.0617
SD,4647.4841,1.780568e+09,8571.1530,0.0109,0.0197,0.0073


In [41]:
# Gradient Boosting Regressor
model_03 = create_model('gbr')
model_03 = tune_model(model_03, fold= 5, n_iter = 100, optimize = 'RMSE', verbose = True, choose_better = True)
tuned_models.append(model_03)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,78476.9793,1.155356e+10,107487.5003,0.8552,0.0690,0.0527
1,77521.7314,9.431574e+09,97116.2897,0.8577,0.0751,0.0562
2,70924.6647,7.736962e+09,87960.0018,0.8388,0.0647,0.0518
3,77215.3467,9.796246e+09,98975.9864,0.8354,0.0705,0.0558
4,80398.2863,1.146660e+10,107082.1935,0.8633,0.0984,0.0646
Mean,76907.4017,9.996988e+09,99724.3943,0.8501,0.0755,0.0562
SD,3191.0883,1.417772e+09,7213.4080,0.0109,0.0119,0.0045


## 2. 앙상블

In [42]:
prediction_models = []

### 배깅

In [43]:
bag_01_15 = ensemble_model(model_01, n_estimators = 15, fold=3, optimize = 'RMSE')
prediction_models.append(bag_01_15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,83591.6416,1.198537e+10,109477.7248,0.8464,0.0777,0.0588
1,75458.5159,8.895774e+09,94317.4124,0.8255,0.0657,0.0535
2,84743.8558,1.253813e+10,111973.7940,0.8313,0.1045,0.0689
Mean,81264.6711,1.113976e+10,105256.3104,0.8344,0.0827,0.0604
SD,4132.4309,1.602703e+09,7801.8036,0.0088,0.0162,0.0064


In [44]:
bag_02_15 = ensemble_model(model_02, n_estimators = 15, fold=5, optimize = 'RMSE')
prediction_models.append(bag_02_15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,82957.6626,1.323819e+10,115057.3433,0.8340,0.0735,0.0561
1,86948.8240,1.168477e+10,108096.1061,0.8237,0.0834,0.0640
2,72945.0890,8.043362e+09,89684.7949,0.8324,0.0647,0.0532
3,84385.4307,1.100252e+10,104892.8861,0.8152,0.0766,0.0615
4,84764.4303,1.330928e+10,115365.8528,0.8413,0.1162,0.0719
Mean,82400.2873,1.145562e+10,106619.3966,0.8293,0.0829,0.0613
SD,4897.5626,1.924315e+09,9377.0111,0.0090,0.0177,0.0065


In [45]:
bag_03_15 = ensemble_model(model_03, n_estimators = 15, fold=3, optimize = 'RMSE')
prediction_models.append(bag_03_15)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,82040.8210,1.180323e+10,108642.6836,0.8488,0.0776,0.0577
1,73635.9110,8.338392e+09,91314.7951,0.8365,0.0647,0.0526
2,83380.5866,1.191695e+10,109164.7778,0.8396,0.1014,0.0674
Mean,79685.7729,1.068619e+10,103040.7522,0.8416,0.0812,0.0592
SD,4312.7226,1.660794e+09,8294.2429,0.0052,0.0152,0.0061


### 부스팅

In [46]:
boo_01 = ensemble_model(model_01, method = 'Boosting', fold=3, optimize = 'RMSE')
prediction_models.append(boo_01)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,82853.7483,1.163560e+10,107868.4451,0.8509,0.0758,0.0583
1,76903.0386,9.056920e+09,95167.8505,0.8224,0.0672,0.0551
2,84607.7037,1.260392e+10,112267.1972,0.8304,0.1020,0.0681
Mean,81454.8302,1.109881e+10,105101.1643,0.8346,0.0817,0.0605
SD,3297.2913,1.496978e+09,7249.8412,0.0120,0.0148,0.0055


In [47]:
boo_02 = ensemble_model(model_02, method = 'Boosting', fold=3, optimize = 'RMSE')
prediction_models.append(boo_02)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,87540.6661,1.297003e+10,113886.0557,0.8338,0.0835,0.0622
1,79092.7852,9.606962e+09,98015.1111,0.8116,0.0719,0.0573
2,86075.2744,1.261698e+10,112325.3140,0.8302,0.0975,0.0678
Mean,84236.2419,1.173132e+10,108075.4936,0.8252,0.0843,0.0624
SD,3685.8471,1.509050e+09,7142.2429,0.0097,0.0105,0.0043


In [48]:
boo_03 = ensemble_model(model_03, method = 'Boosting', fold=3, optimize = 'RMSE')
prediction_models.append(boo_03)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,85152.7293,1.229781e+10,110895.5030,0.8424,0.0787,0.0599
1,74131.7117,8.570155e+09,92575.1312,0.8319,0.0678,0.0537
2,82566.9204,1.173368e+10,108322.1317,0.8421,0.0964,0.0654
Mean,80617.1205,1.086722e+10,103930.9220,0.8388,0.0810,0.0597
SD,4705.8118,1.640514e+09,8098.1911,0.0049,0.0118,0.0048


### 스태킹

In [49]:
stactking = stack_models(estimator_list=[model_01, model_02, model_03],  meta_model = model_02, fold=5, optimize='RMSE')
prediction_models.append(stactking)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,79920.4134,1.204853e+10,109765.8180,0.8490,0.0719,0.0547
1,84603.5140,1.118353e+10,105752.1990,0.8312,0.0780,0.0604
2,74214.3827,8.520020e+09,92303.9533,0.8225,0.0661,0.0539
3,84574.1067,1.130527e+10,106326.2580,0.8101,0.0766,0.0609
4,84739.4888,1.266786e+10,112551.5834,0.8490,0.1040,0.0686
Mean,81610.3811,1.114504e+10,105339.9623,0.8324,0.0793,0.0597
SD,4125.3027,1.417968e+09,6966.7198,0.0151,0.0130,0.0053


### 블렌딩

In [50]:
blended = blend_models(estimator_list = best_3, fold = 5)
prediction_models.append(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,76395.7651,1.085058e+10,104166.1003,0.8640,0.0674,0.0516
1,75384.0960,8.840476e+09,94023.8040,0.8666,0.0722,0.0556
2,68758.2082,7.076603e+09,84122.5455,0.8526,0.0612,0.0499
3,78592.2542,1.018822e+10,100936.7065,0.8289,0.0773,0.0582
4,78777.2767,1.088654e+10,104338.5725,0.8702,0.1015,0.0647
Mean,75581.5200,9.568482e+09,97517.5457,0.8564,0.0759,0.0560
SD,3647.9292,1.449453e+09,7668.8018,0.0150,0.0138,0.0052


## 3. 모델 튜닝 결과

In [51]:
for model in tuned_models:
    print(model.__class__.__name__)
    display(predict_model(model))

ExtraTreesRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,71282.9869,9.259231e+09,96224.8958,0.8731,0.0718,0.0526


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1331576.0
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1315486.5
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1807107.5
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1853411.5
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1123882.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1140311.0
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1850125.5
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1300196.5
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1502461.5


RandomForestRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,76767.2128,1.118869e+10,105776.6149,0.8467,0.0847,0.0564


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1358031.5
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1321255.0
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1829819.0
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1855293.5
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1120403.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1143870.5
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1860986.0
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1291819.0
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1527364.0


GradientBoostingRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Gradient Boosting Regressor,71003.0868,9.471983e+09,97324.1154,0.8702,0.0792,0.0521


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1.366138e+06
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1.323531e+06
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1.860721e+06
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1.874561e+06
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1.144701e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1.121303e+06
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1.859667e+06
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1.288729e+06
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1.512330e+06


In [52]:
for model in prediction_models:
    print(model.__class__.__name__)
    display(predict_model(model))

BaggingRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,73639.9413,9.846972e+09,99231.9079,0.865,0.0765,0.0546


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1.365367e+06
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1.312902e+06
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1.801648e+06
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1.868096e+06
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1.148063e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1.126908e+06
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1.845298e+06
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1.307967e+06
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1.482089e+06


BaggingRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,75240.3754,1.057704e+10,102844.7614,0.855,0.0814,0.0555


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Stacking Regressor,75669.0222,1.072889e+10,103580.3373,0.853,0.0844,0.0556


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1386740.0
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1303826.5
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1838701.0
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1854782.0
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1166019.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1134532.5
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1869713.0
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1300011.0
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1502040.5


VotingRegressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,67594.5145,8.409842e+09,91705.1888,0.8847,0.0694,0.0496


,total_pop,jeju_resd_pop,card_use_amt,delivery_amt,market_amt,card_use_cnt,delivery_cnt,market_cnt,em_cnt,visit_pop,work_pop,youth_pop,total_frgn_pop,em_g,Label
0,9620.0,5616.918945,71071000.0,2063409.0,21205494.0,2255.0,102.0,632.0,692.0,167639.171875,23541.677734,779.403625,39.449707,1295200.0,1.347138e+06
1,9496.0,5610.537109,59958200.0,2244510.0,14768274.0,1918.0,77.0,495.0,617.0,174364.031250,22105.021484,779.149963,45.676476,1382650.0,1.332885e+06
2,10031.0,5800.811523,76134688.0,1021246.0,32159648.0,2209.0,30.0,662.0,905.0,173278.875000,26145.664062,846.000610,71.215790,1777500.0,1.810345e+06
3,10033.0,6150.803223,84039672.0,389272.0,27035286.0,2282.0,15.0,672.0,943.0,159973.156250,19882.634766,821.375916,40.458080,1861450.0,1.856456e+06
4,9148.0,6560.730469,58093732.0,4103151.0,13165370.0,1907.0,183.0,482.0,577.0,157236.531250,26164.310547,863.184204,14.443357,1146350.0,1.135808e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,9127.0,6863.645996,73020672.0,5338413.0,13773715.0,2056.0,227.0,544.0,551.0,157295.671875,27090.720703,855.767517,19.981483,1108450.0,1.110523e+06
379,10033.0,6479.880859,64687268.0,596796.0,16944842.0,2060.0,15.0,573.0,925.0,168689.703125,28017.539062,923.685364,53.184963,1799700.0,1.830232e+06
380,9620.0,5961.879395,71100936.0,990420.0,12748164.0,2111.0,56.0,510.0,617.0,162938.062500,23213.777344,767.062134,40.499687,1129500.0,1.283614e+06
381,9668.0,5904.659180,64202240.0,1626605.0,18211764.0,2324.0,51.0,697.0,734.0,170114.890625,24213.060547,825.964905,70.591194,1467300.0,1.523723e+06


In [53]:
final_01 = finalize_model(model_03) # 성능 상위모델 1번
final_02 = finalize_model(boo_03) # 성능 상위모델 2번
final_03  = finalize_model(blended)

# 최종모델 결정 및 성능 평가

In [56]:
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import mean_squared_error

value_list = [] # test 용 복사
for model in [final_01, final_02, final_03]:
    prediction_i = predict_model(model, data = X_val)
    y_pred_i = prediction_i['Label']
    
    value_list.append(y_pred_i)
    
    y = y_val
    aucscore = mean_squared_error(y, y_pred_i,squared=False)
    print("RMSE: {}".format(aucscore))

RMSE: 50558.49080095657
RMSE: 43831.50112890873
RMSE: 34630.74900097417
